# **New scraping from DE**

# Imports

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib
#import urllib3.request,sys,time
from urllib.error import HTTPError
from urllib.request import urlopen
import pandas as pd
import time
import warnings
#from google.colab import files
warnings.filterwarnings('ignore')

# Treatment functions

In [2]:
#récupère les articles présents dans le résultat de recherche du DE
def getArticle(pageurl):
    try:
        html = urlopen(pageurl)
    except HTTPError as e:
        return None
    try:
        headers = {
            'User-Agent': 'My User Agent 1.0',
        }
        res= requests.get(pageurl, headers=headers)
        bsObj = BeautifulSoup(res.text,'lxml')
        #bsObj = BeautifulSoup(html.read())
        item = bsObj.find("div",{'class':{'cf'}}).findAll('a')
    except AttributeError as e:
        return None
    articles_list=[]
    for j in item:#one item is one article
        article=dict()
        article['url']="www.express.co.uk"+j.get('href')
        article['date']=j.p.time['datetime']
        articles_list.append(article)
    return articles_list

In [3]:
#crée un dictionnaire avec les infos de l'article
def getNews(article_url):
    try:
        html = urlopen(article_url)
    except HTTPError as e:
        print(article_url)
        return None
    bsObj = BeautifulSoup(html.read())
    article=dict()
    try:
        article["author"]=bsObj.find("div",{'class':{'author'}}).text
    except AttributeError as e:
        article["author"]=''
    try:
        article["headline"]=bsObj.find("header",{'class':{'clearfix'}}).h1.text
    except AttributeError as e:
        article["headline"]=''
    try:
        article["summary"]=bsObj.find("header",{'class':{'clearfix'}}).h3.text
    except AttributeError as e:
        article["summary"]=''
    try:
        article["date"]=bsObj.find("div",{'class':{'dates'}}).time["datetime"][:10]
    except AttributeError as e:
        article["date"]= ''
    try:
        paras=bsObj.find_all("div",{'class':{'text-description'}})
        fulltext=""
        for para in paras:
            paras_text=" \n ".join(i.text for i in para.find_all("p"))
            fulltext=fulltext+paras_text
        article["fulltext"]=fulltext
    except AttributeError as e:
        article["fulltext"]= ''
    article["url"]=article_url
    time.sleep(1)
    return article

In [4]:
#transforme la liste d'url en dataframe avec les articles
def get_articles_from_links(articles_links):
    articles = []
    for ind in articles_links.index :
        if ind%10==0:
            print(ind)
        article_url='https://'+articles_links['url'][ind]
        news=getNews(article_url)
        if news is not None:
            articles.append(news)
        time.sleep(1)
    return articles

In [5]:
def articles_from_base_url_DE(base_url):
    #parcours de tous les articles dispos
    url=[]
    for i in range(0,10001,10): #Doit aller jusqu'à #10001 !! je raccourcis pour les tests
        if i%1000==0:
            print(i)
        url.append(base_url+str(i))
    articles_links = pd.DataFrame()
    #création de la database avec tous les liens d'articles
    for i in range(len(url)):
        inter = pd.DataFrame(getArticle(url[i]))
        articles_links = pd.concat([articles_links, inter], ignore_index=True, axis=0)
    news = get_articles_from_links(articles_links)
    df_news = pd.DataFrame(news)
    return df_news

# Tries with Facebook

In [ ]:
#URL de base
base_url="https://www.express.co.uk/search?s=facebook&order=oldest&o="

#parcours de tous les articles dispos
fb_url=[]
for i in range(0,21,10): #Doit aller jusqu'à #10001 !! je raccourcis pour les tests
    fb_url.append(base_url+str(i))

In [ ]:
articles_links = pd.DataFrame()

In [ ]:
#création de la database avec tous les liens d'articles
for i in range(len(fb_url)):
    inter = pd.DataFrame(getArticle(fb_url[i]))
    articles_links = pd.concat([articles_links, inter], ignore_index=True, axis=0)

In [ ]:
articles_links

,url,date
0,www.express.co.uk/news/uk/3045/Father-won-t-re...,2007-03-29T01:00:00Z
1,www.express.co.uk/news/uk/6664/Teacher-jailed-...,2007-05-10T17:45:07Z
2,www.express.co.uk/news/uk/8422/One-in-five-chi...,2007-05-31T01:00:00Z
3,www.express.co.uk/news/uk/8840/Man-remanded-on...,2007-06-05T02:15:02Z
4,www.express.co.uk/news/uk/8889/Man-charged-wit...,2007-06-05T01:00:00Z
5,www.express.co.uk/news/uk/8970/Kate-date-murde...,2007-06-06T01:00:00Z
6,www.express.co.uk/sport/othersport/12517/Doubl...,2007-07-06T01:00:00Z
7,www.express.co.uk/news/uk/14609/Facebook-found...,2007-07-25T08:45:10Z
8,www.express.co.uk/news/uk/14969/Eugenie-in-row...,2007-07-29T01:00:00Z
9,www.express.co.uk/news/uk/15068/New-crackdown-...,2007-07-30T19:15:07Z


In [ ]:
news = get_articles_from_links(articles_links)

0


KeyboardInterrupt: ignored

In [ ]:
df_news = pd.DataFrame(news)

Le problème est qu'on a encore énormmément trop d'articles, à cause de toutes les pubs sur les artciles qui mentionnent facebook. On ne va garder que les articles dont l'url contient le mort "facebook".

In [ ]:
df_fb_DE = df_news[df_news['url'].str.contains('Facebook|Meta|facebook')]

# Dataframe Facebook

On systématise le traitement avec une fonction générale dans la partie fonctions

In [6]:
#URL de base
base_url="https://www.express.co.uk/search?s=facebook&order=oldest&o="

In [ ]:
df_news_fb = articles_from_base_url_DE(base_url)

Attention pour le DE les majuscules dans l'url importent !!

In [10]:
df_news_fb['url'] = df_news_fb['url'].str.lower()

In [11]:
df_fb_DE = df_news_fb[df_news_fb['url'].str.contains('meta|facebook|messenger|instagram|oculus|whatsApp|zuckerberg|olivan|clegg|social-media|gafa|gafam|big-tech|consumer-privacy|tech')]

In [ ]:
df_fb_DE.to_csv('df_fb_DE.csv')
#files.download('df_fb_DE.csv')

# Dataframe Amazon

In [6]:
#URL de base
base_url="https://www.express.co.uk/search?s=amazon&order=oldest&o="

In [ ]:
df_news_am = articles_from_base_url_DE(base_url)

In [ ]:
df_news_am['url'] = df_news_am['url'].str.lower()

In [ ]:
df_am_DE = df_news_am[df_news_am['url'].str.contains('amazon|prime|whole-foods|zappos|pillpack|twitch|audible|goodreads|imdb|bezos|jassy|olsavsky|alexander|gafa|gafam|big-tech|consumer-privacy|tech')]

In [ ]:
df_am_DE

-> piste d'amélioration : aussi considérer les artciles concernnat des compagnies directement liées aux BigTechs, par exemple Twitch pour Amazon etc. Aussi leurs dirigeants ?
Idée : 5 produits principaux, 5 sociétés annexes, 5 dirigeants (tout au maximum)
Car l'url peut ne pas contenir directement le nom de la BigTech, bien qu'elle soit très concernée.

Mais d'après le stage précédent, il nous faut ici enlever les articles en lien avec la forêt amazonienne, qui reviennent beaucoup.

In [ ]:
yy=df_am_DE[df_am_DE['fulltext'].str.contains('forest|tribe|jungle|swallows-and-amazon|amazons')]
df_am_DE.drop(yy.index,inplace=True)

In [ ]:
df_am_DE.to_csv('df_am_DE.csv')
#files.download('df_am_DE.csv')

# Dataframe Apple

In [ ]:
#URL de base
base_url="https://www.express.co.uk/search?s=apple&order=oldest&o="

In [ ]:
df_news_ap = articles_from_base_url_DE(base_url)

In [ ]:
df_news_ap['url'] = df_news_ap['url'].str.lower()

In [ ]:
df_ap_DE = df_news_ap[df_news_ap['url'].str.contains('apple|iphone|ipad|mac|watch|macbook|ios|airpods|app-store|itunes|icloud|beats|siri|shazam|cook|jobs|gafa|gafam|big-tech|consumer-privacy|tech')]

In [ ]:
yy=df_ap_DE[df_ap_DE['fulltext'].str.contains('apples|pie|pies|recipe|tart|crumble|bake|vinegar')]
df_ap_DE.drop(yy.index,inplace=True)

In [ ]:
df_ap_DE.to_csv('df_ap_DE.csv')
#files.download('df_ap_DE.csv')

# DataFrame Google

In [10]:
#URL de base
base_url="https://www.express.co.uk/search?s=google&order=oldest&o="

In [ ]:
df_news_go = articles_from_base_url_DE(base_url)

In [12]:
df_news_go['url'] = df_news_go['url'].str.lower()

In [13]:
df_go_DE = df_news_go[df_news_go['url'].str.contains('google|android|chrome|gmail|maps|playstore|pixel|waze|youTube|alphabet|mandiant|fitbit|looker|nest|doubleclick|page|brin|pichai|kurian|gafa|gafam|big-tech|consumer-privacy|tech')]

In [14]:
df_go_DE.to_csv('df_go_DE.csv')
#files.download('df_go_DE.csv')

# Dataframe Microsoft

In [15]:
#URL de base
base_url="https://www.express.co.uk/search?s=microsoft&order=oldest&o="

In [ ]:
df_news_mi = articles_from_base_url_DE(base_url)

In [17]:
df_news_mi['url'] = df_news_mi['url'].str.lower()

In [18]:
df_mi_DE = df_news_mi[df_news_mi['url'].str.contains('microsoft|windows|cortana|excel|explorer|office|edge|teams|outlook|powerpoint|skype|surface|word|xbox|linkedIn|github|mojang|gates|nadella|gafa|gafam|big-tech|consumer-privacy|tech')]

In [21]:
df_mi_DE.to_csv('df_mi_DE.csv')
#files.download('df_mi_DE.csv')

# Dataframe final

In [47]:
df_fb_DE = pd.read_csv('df_fb_DE.csv', index_col=[0])
df_am_DE = pd.read_csv('df_am_DE.csv', index_col=[0])
df_ap_DE = pd.read_csv('df_ap_DE.csv', index_col=[0])

In [48]:
#from google.colab import files
df_final = pd.concat([df_fb_DE, df_am_DE, df_ap_DE, df_go_DE, df_mi_DE]).reset_index()
df_final.drop_duplicates(subset ="url", keep = 'first', inplace=True)
df_final.to_csv('articlesDE.csv')
#files.download('articlesDE.csv')